# GPT-2 based letters generation

In [1]:
%pip install gpt-2-simple
%pip install tensorflow==1.14
%pip install tensorflow-gpu==1.14.0

  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=23580 sha256=7297a7659635672823caf73a6a04df81874cfcce7f760032057d8cdeea1c138f
  Stored in directory: /root/.cache/pip/wheels/0c/f8/23/b53ce437504597edff76bf9c3b8de08ad716f74f6c6baaa91a
Successfully built gpt-2-simple
     |████████████████████████████████| 109.2MB 93kB/s 
     |████████████████████████████████| 3.2MB 53.5MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 491kB 50.3MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 377.0MB 42

## Download GPT-2 model

In [2]:
import os
import sys
import requests
from tqdm import tqdm

def download_model(model="124M"):
  """Downloads a specific model.
    See https://github.com/minimaxir/gpt-2-simple#usage.
  """
  subdir = os.path.join('models', model)
  if not os.path.exists(subdir):
      os.makedirs(subdir)

  for filename in ['checkpoint','encoder.json','hparams.json','model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta', 'vocab.bpe']:
      r = requests.get("https://openaipublic.blob.core.windows.net/gpt-2/" + subdir + "/" + filename, stream=True)
      with open(os.path.join(subdir, filename), 'wb') as f:
          file_size = int(r.headers["content-length"])
          chunk_size = 1000
          with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
              for chunk in r.iter_content(chunk_size=chunk_size):
                  f.write(chunk)
                  pbar.update(chunk_size)

## Train model

In [3]:
import gpt_2_simple as gpt2
import os
import requests

model_name = "124M"
download_model(model=model_name)

# provide file for training
file_name = "input_letters"

# Resets model on re-finetune
import tensorflow as tf
tf.reset_default_graph()

sess = gpt2.start_tf_sess()
gpt2.finetune(
    sess,
    file_name,
    model_name=model_name,
    steps=1000,
    print_every=25,
    sample_every=200,
    save_every=500
)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.89it/s]


dataset has 106901 tokens
Training...
[25 | 40.10] loss=2.58 avg=2.58
[50 | 70.60] loss=1.17 avg=1.87
[75 | 101.06] loss=0.24 avg=1.32
[100 | 131.54] loss=0.08 avg=1.01
[125 | 162.01] loss=0.06 avg=0.81
[150 | 192.48] loss=0.04 avg=0.68
[175 | 222.96] loss=0.05 avg=0.59
[200 | 253.43] loss=0.04 avg=0.52
======== SAMPLE 1 ========
 Chicago — and that it is perhaps much to our advantage that we should be attended, not by envy, but by an immense 
affection: for which reason I cannot be idle, being very much indebted to you: but when you have completed the business of your own accord, I can offer myself 
to your own advantage — that is to say, my gratitude indeed: for having done so, I have been enabled by your own merits to be enriched by yours — 
and that, indeed, I am a little astonished that you should desire a right in my debt — more than can be due both to you and me, or 
to both: for it seems to me that you owe me both, both.
If, however, such an objection were not so very strong, b

## Generate new letters using trained model

In [4]:
generated = gpt2.generate(
    sess,
    length=250,
    temperature=0.7,
    prefix="To my one and only...",
    return_as_list=True,
    run_name='run1'
)

print(generated)

['To my one and only...\nRichard Steele==============\nMy heart overflows with emotion and joy! I do not know what heavenly languor, what infinite pleasure permeates it and burns me up. Oh! let me repeat that name a hundred times, a thousand times over; for three days now it has lived within me, oppressed me, \nset me afire. I am not writing to you, no, I am close beside you. I see you, I hear you. Eternity in your arms… Heaven, Hell, everything, \nall is within you, redoubled… Oh! Leave me free to rave in my delirium. Drab, tame, constricting reality is no longer enough for me. \nWe must live our lives to the full, loving and suffering to extremes!…==============\nMy dear Friend,\nYes, I have told you, and repeat it: I love you dearly. You certainly said the same thing to me, I begin to know the world.\nI will tell you what I suggest, now: pay attention. I don’t want to remain a shopgirl, but a little more my own mistress, and \nwould therefore like to find someone to keep me.\n']


In [5]:
import json

prefixes = None
with open("generate_with_prefixes", "r") as prefixes_file:
  prefixes = json.loads(prefixes_file.read())

assert prefixes is not None

generated = []
for prefix in prefixes:
  g = gpt2.generate(
      sess,
      length=250,
      temperature=0.7,
      prefix=prefix,
      return_as_list=True,
      run_name='run1'
  )
  generated.append({prefix: g[0]})
  print('generated', generated[-1])

print('generation finished.')

generated {'My dearest Teresa, I have read this book in your garden;–my love, you were absent, or else I could not have': 'My dearest Teresa, I have read this book in your garden;–my love, you were absent, or else I could not have", "I received your letter my ever dearest Maria,–my dearest Maria, what a strange charm you have here–are all parts", "My dear Girl, my dearest Teresa, I have become so attached to you that I cannot even begin to express an absolute love between the two of us", "To Robert Browning: My dearest Robert, I have been a walk this morning with my heart meditating. This is no time for writing or doing anything, I must go to bed.", "My darling, I am now nearly at an end of my volition. Do you remember that hot \nHim that you sent for the day? I told you Monday I was going to take a walk with my heart. Tuesday I was going to take a walk with my hair. Wednesday I was going to take a walk with my shoes.\nI was going to take a walk with my cat, who was going to Jeannette,

In [6]:
with open("generated", "w") as f:
  json.dump(generated, f)